In [1]:
!pip install pyspark
!pip install python-dotenv
!pip install snowflake-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 8.6 MB/s eta 0:00:00


In [2]:
# Step 1: Initialize variables

import snowflake.connector
from google.colab import userdata

# Replace the placeholders with your actual Snowflake credentials
conn = snowflake.connector.connect(
    user = userdata.get('SNOWFLAKE_USER'),
    password = userdata.get('SNOWFLAKE_PASSWORD'),
    account= userdata.get('SNOWFLAKE_ACCOUNT'),
    warehouse='COMPUTE_WH',
    database='BIGDATA_GITHUB',
    schema='RAW',
    role='ACCOUNTADMIN'
)

In [3]:
# Step 2: Create Snowflake stage for parquet file created by analytics

cur = conn.cursor()
url = 'azure://matthewleffler1.blob.core.windows.net/kaggle-datasets/clean_data/Influence_Top/'

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
        CREATE OR REPLACE STAGE BIGDATA_GITHUB.ANALYTICS.azure_parquet_stage_Influence_Top
          URL = '{url}'
          CREDENTIALS = (
            AZURE_SAS_TOKEN = '{userdata.get('AZURE_SAS_TOKEN')}'
          )
          FILE_FORMAT = (TYPE = PARQUET);
        """)
    cur.execute("COMMIT;")
    print(f"Successfullt created stage.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()

Successfullt created stage.


In [4]:
# Step 3: Create Snowflake table defination based on parquet analytics

cur = conn.cursor()
table_name = "BIGDATA_GITHUB.ANALYTICS.Influence_Top_100"

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      CREATE OR REPLACE TABLE {table_name} (
      user_id          BIGINT,
      user_login       STRING,
      follower_count   BIGINT,
      total_stars      BIGINT,
      total_commits    BIGINT,
      total_forks      BIGINT,
      influence_score  FLOAT
      );
          """)
    cur.execute("COMMIT;")
    print("Table created successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error creating database object: {e}")
finally:
    cur.close()

Table created successfully.


In [5]:
# Step 4: Load data into Snowflake table

cur = conn.cursor()

try:
    cur.execute("BEGIN;")
    cur.execute(f"""
      COPY INTO {table_name}
      FROM @azure_parquet_stage_Influence_Top
      FILE_FORMAT = (TYPE = PARQUET)
      MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
      ON_ERROR = CONTINUE;
          """)
    cur.execute("COMMIT;")
    print(f"Data loaded into {table_name} successfully.")
except Exception as e:
    cur.execute("ROLLBACK;")
    print(f"Error loading data: {e}")
finally:
    cur.close()

Data loaded into BIGDATA_GITHUB.ANALYTICS.Influence_Top_100 successfully.
